<a href="https://colab.research.google.com/github/elpados/subsidy-calculator/blob/main/ess_subsidy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tabulate import tabulate

def calculate_subsidy(expense_data, business_size, pv_capacity, ess_capacity, pv_cost_per_kw, other_expenses):
    """
    Calculate subsidy for the eligible expenses.

    :param expense_data: Dictionary containing all eligible expenses.
    :param business_size: The size of the business ('Large', 'Medium', 'Small').
    :param pv_capacity: Capacity of the photovoltaic system in kWp.
    :param ess_capacity: Capacity of the energy storage system in kWh.
    :param pv_cost_per_kw: Cost per kW of PV system (€).
    :param other_expenses: Dictionary containing other expenses (e.g., study/design, consulting fees, etc.).

    :return: A dictionary with the calculated subsidies and total system cost.
    """

    # Subsidy percentages based on business size
    subsidy_percentages = {
        "Large": 0.30,
        "Medium": 0.40,
        "Small": 0.50
    }

    # Validate inputs
    if pv_capacity <= 0 or ess_capacity <= 0 or pv_cost_per_kw <= 0:
        raise ValueError("PV capacity, ESS capacity, and PV cost per kW must be positive numbers.")

    # Calculate hours of storage
    hours = ess_capacity / pv_capacity

    # Maximum subsidies per MWh based on hours
    max_battery_subsidy_1h = 400000  # for 1-hour battery
    max_battery_subsidy_2h = 350000  # for 1-2 hour battery

    # Determine max subsidy per MWh
    if hours <= 1:
        max_allowed_subsidy_per_mwh = max_battery_subsidy_1h
    elif 1 < hours <= 2:
        max_allowed_subsidy_per_mwh = max_battery_subsidy_2h
    else:
        raise ValueError("Hours of storage must be between 0 and 2.")

    # Calculate the subsidy for main expenses
    main_expenses = expense_data.get('main_expenses', 0)
    max_allowed_subsidy = max_allowed_subsidy_per_mwh * hours
    main_expenses_subsidy = min(main_expenses, max_allowed_subsidy) * subsidy_percentages[business_size]

    # Calculate PV cost
    pv_cost = pv_capacity * pv_cost_per_kw

    # Calculate subsidy for supportive expenses
    supportive_expenses = {
        'study': other_expenses.get('study', 0),
        'consulting': other_expenses.get('consulting', 0),
        'interconnection': other_expenses.get('interconnection', 0)
    }

    # Subsidy for study/design (50% of expense)
    study_subsidy = min(supportive_expenses['study'] * 0.50, 20000)

    # Subsidy for consulting (100% of expense)
    consulting_subsidy = min(supportive_expenses['consulting'] * 1.00, 15000)

    # Subsidy for interconnection (50% of expense)
    interconnection_subsidy = min(supportive_expenses['interconnection'] * 0.50, 5000)

    # Calculate total supportive expenses
    total_supportive_expenses = supportive_expenses['study'] + supportive_expenses['consulting'] + supportive_expenses['interconnection']

    # Deduct subsidies from ESS costs
    total_ess_cost = main_expenses + total_supportive_expenses
    total_subsidies = main_expenses_subsidy + study_subsidy + consulting_subsidy + interconnection_subsidy
    total_ess_cost_less_subsidy = total_ess_cost - total_subsidies

    # Calculate total system cost including PV cost
    total_system_cost_less_subsidy = total_ess_cost_less_subsidy + pv_cost

    return {
        "main_expenses_subsidy": main_expenses_subsidy,
        "pv_cost": pv_cost,
        "study_subsidy": study_subsidy,
        "consulting_subsidy": consulting_subsidy,
        "interconnection_subsidy": interconnection_subsidy,
        "total_supportive_expenses": total_supportive_expenses,
        "total_ess_cost": total_ess_cost,
        "total_ess_cost_less_subsidy": total_ess_cost_less_subsidy,
        "total_system_cost_less_subsidy": total_system_cost_less_subsidy
    }


# Input from user
print("Subsidy Calculator")
business_size = input("Enter the size of the business (Large, Medium, Small): ").capitalize()

pv_capacity = float(input("Enter the photovoltaic system capacity in kWp (e.g., 100): "))
ess_capacity = float(input("Enter the energy storage system capacity in kWh (e.g., 200): "))
pv_cost_per_kw = float(input("Enter the cost per kW of the PV system (€): "))

main_expenses = float(input("Enter the main expenses for the energy storage system (€): "))
study_expense = float(input("Enter the expense for study/design (€): "))
consulting_expense = float(input("Enter the consulting fees (€): "))
interconnection_expense = float(input("Enter the interconnection cost (€): "))

# Create dictionaries for inputs
expense_data = {'main_expenses': main_expenses}
other_expenses = {
    'study': study_expense,
    'consulting': consulting_expense,
    'interconnection': interconnection_expense
}

# Calculate subsidy
try:
    subsidy_results = calculate_subsidy(
        expense_data, business_size, pv_capacity, ess_capacity, pv_cost_per_kw, other_expenses
    )

    # Prepare the summary table
    table = [
        ["Main Expenses Subsidy (ESS)", f"€{subsidy_results['main_expenses_subsidy']:.2f}"],
        ["PV System Cost", f"€{subsidy_results['pv_cost']:.2f}"],
        ["Study Subsidy", f"€{subsidy_results['study_subsidy']:.2f}"],
        ["Consulting Subsidy", f"€{subsidy_results['consulting_subsidy']:.2f}"],
        ["Interconnection Subsidy", f"€{subsidy_results['interconnection_subsidy']:.2f}"],
        ["Total Supportive Expenses", f"€{subsidy_results['total_supportive_expenses']:.2f}"],
        ["Total ESS Cost (Main + Supportive)", f"€{subsidy_results['total_ess_cost']:.2f}"],
        ["Total ESS Cost After Subsidy", f"€{subsidy_results['total_ess_cost_less_subsidy']:.2f}"],
        ["Total System Cost After Subsidy (ESS + PV)", f"€{subsidy_results['total_system_cost_less_subsidy']:.2f}"]
    ]

    # Print the summary table
    print("\nSummary of Costs and Subsidies:")
    print(tabulate(table, headers=["Description", "Amount"], tablefmt="grid"))

except ValueError as e:
    print(f"Error: {e}")


Subsidy Calculator
Enter the size of the business (Large, Medium, Small): small
Enter the photovoltaic system capacity in kWp (e.g., 100): 100
Enter the energy storage system capacity in kWh (e.g., 200): 200
Enter the cost per kW of the PV system (€): 600
Enter the main expenses for the energy storage system (€): 90000
Enter the expense for study/design (€): 0
Enter the consulting fees (€): 0
Enter the interconnection cost (€): 0

Summary of Costs and Subsidies:
+--------------------------------------------+------------+
| Description                                | Amount     |
+============================================+============+
| Main Expenses Subsidy (ESS)                | €45000.00  |
+--------------------------------------------+------------+
| PV System Cost                             | €60000.00  |
+--------------------------------------------+------------+
| Study Subsidy                              | €0.00      |
+--------------------------------------------+-------